In [1]:
#import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

### Processing steps

##### Denoising (Adaptive non-local means)

In [2]:
###### adaptive NLM #####
def denoise(img, h=10, window=100, patch=7):
    result = cv2.fastNlMeansDenoising(img, None, h=h, templateWindowSize=patch, searchWindowSize=window)

    return result

##### Contrast enhancement (custom)

In [3]:
############# contrast enhancement ##################
def contrast_enhancement(img, threshold=30, box=(5,5)):
    # shape of input image
    rows, cols = img.shape

    #the result array  #same shape as input
    result = img.copy()

    # get width and height of the patch (box)
    box_width, box_height = box[1], box[0]

    # denoise the image one patch at a time
    for y in range(0, rows, box_height):
        for x in range(0, cols, box_width):
            # Extract the patch
            patch = img[y:y+box_height, x:x+box_width]

            # Calculate the average patch value
            avg = np.mean(patch[:,:])

            # if the patch average is considered black
            if avg <= threshold:
                # make the black pixels blacker
                patch[patch <= threshold] = patch[patch <= threshold]*(0.8)
                # make the non-black pixels brighter
                patch[patch > threshold] = avg*(1.1)
            #else:
                # make all pixels avg
                #patch[:, :] = avg*(1.1)

            # Update the patch in the result image
            result[y:y+box_height, x:x+box_width] = patch

    return result

##### Blurring (Gaussian blurr)

In [4]:
#Gaussian blurr
def blurr(img, kernel=(7,7)):
    denoised_image = cv2.GaussianBlur(img, kernel, 0) #apply Gaus. blur
    return denoised_image

##### Processing 

For black patch: make black pixels darker than the patch avg, and the non-black pixels brighter than avg.
For non-black patch : make all pixels slightly brighter than the patch average.


In [5]:
def process(img, threshold=25, box=(3,5)):
    # shape of input image
    rows, cols = img.shape

    #the result array  #same shape as input
    result = np.zeros((rows, cols))

    # get width and height of the patch (box)
    box_width, box_height = box[1], box[0]

    # process the image one patch at a time
    for y in range(0, rows, box_height):
        for x in range(0, cols, box_width):
            # Extract the patch
            patch = img[y:y+box_height, x:x+box_width]

            # Calculate the average patch value
            avg = np.mean(patch[:,:])

            # if the patch average is considered black
            if avg <= threshold:
                # make the black pixels blacker
                patch[patch <= threshold] = avg*(0.55)
                # make the non-black pixels brighter
                patch[patch > threshold] = avg*(1.25)
            else:
                # make all pixels avg
                patch[:, :] = avg*(1.05)

            # Update the patch in the result image
            result[y:y+box_height, x:x+box_width] = patch

    return result

### Processing functions

##### Process 1 image

In [6]:
def process_image(img):
    temp = denoise(img)
    temp = process(temp, threshold=25, box=(3,5))
    temp = blurr(temp, (3,3))
    result_image = contrast_enhancement(temp, threshold=10, box=(5,5))

    return result_image

##### Process a dataset

In [8]:
################ both .tif and jpeg #################

# dataset = path of dataset
# dest_path = where output will be saved
def process_data(dataset, dest_path):
    # if dest folder doesnt exist, create it
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    # List of all files in the dataset
    image_files = os.listdir(dataset) #name of img file
    tot = len(image_files) 

    count = 0
    for image_file in image_files:
        # full path of the img = dataset path + image file name
        image_path = os.path.join(dataset, image_file)

        # Read the image as numpy array
        image = cv2.imread(image_path)

        # Process the image
        output_img = process_image(image)

        # full path of output = dest folder path + image file name
        output_path = os.path.join(dest_path, image_file)

        # Save the image
        cv2.imwrite(output_path, output_img)

        count+=1
        print(f"{count}/{tot}")
    print("All images processed succesfully.\n")


In [9]:
####################### jpeg only ###################

def process_data_jpeg_only(dataset, dest_path):
    # if dest folder doesnt exist, create it
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)

    # List of all files in the dataset
    image_files = os.listdir(dataset) #name of img file
    tot = len(image_files)/2 # div by 2 as dataset contains .tif and .jpeg both 

    count = 0
    for image_file in image_files:
        # Filter only JPEG images
        if image_file.lower().endswith((".jpg", ".jpeg")):
            # full path of the img = dataset path + image file name
            image_path = os.path.join(dataset, image_file)

            # Read the image as numpy array
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Process the image
            output_img = process_image(image)

            # full path of output = dest folder path + image file name
            output_path = os.path.join(dest_path, image_file)

            # Save the image
            cv2.imwrite(output_path, output_img)

            count+=1
            print(f"{count}/{tot}")
    print("All images processed succesfully.\n")


### Combine images into a single composite image

##### pre-aligned images

In [10]:
# Function to combine aligned images and save the result
def combine_aligned_images(dataset, dest_path):
    # List all JPEG files in the image folder
    image_files = [f for f in os.listdir(dataset) if f.lower().endswith(('.jpg', '.jpeg'))]
    tot = len(image_files)

    # Load the first image to get dimensions
    test_img = cv2.imread(os.path.join(dataset, image_files[0]), cv2.IMREAD_GRAYSCALE)
    # initialise the composite image
    combined_image = np.zeros_like(test_img, dtype=np.float32)

    # Process and combine images
    count = 0
    for image_file in image_files:
        #read and process the image
        img_path = os.path.join(dataset, image_file)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        processed_img = process_image(img)

        #add processed image to the composite image
        combined_image += processed_img

        count +=1
        print(f"{count}/{tot}")

    # Normalize and convert to uint8 format
    combined_image = (combined_image/tot).astype(np.uint8)

    # Save the combined image
    cv2.imwrite(dest_path, combined_image)
    print(f"Composite image saved to {dest_path}")



### Testing

##### process a single image

In [7]:
path = ".\\testing diff methods\\originals\\original.jpeg"
img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
 
result = process_image(img)

cv2.imwrite("output.jpg", result)

True

##### process a dataset

In [12]:
dataset = ".\\test"
dest_path = ".\\output" 

process_data_jpeg_only(dataset, dest_path)

1/3.0
2/3.0


KeyboardInterrupt: 

##### create composite image

In [ ]:
# Specify your image folder and destination path
#dataset = ".\\input (1-20)"
dataset = ".//test"
composite_image = 'output.jpg'

# Call the function to combine and save the images
combine_aligned_images(dataset, composite_image)

1/6
2/6
3/6
4/6
5/6
6/6
Composite image saved to output.jpg
